Copyright (c) Microsoft Corporation.<br>
Licensed under the MIT License.

# 5. Deploy OpenVINO Model to the Percept DK

In this notebook we will:
- Deploy the model to the Percept Dev Kit through the Azure Portal

## Prerequisites
- Converted OpenVINO model (`blob` format)
- Azure Storage account 


In [ ]:
import sys
import os
import shutil

## Package the OpenVINO model as a zip file and upload to blob storage

Create a directory and the following files to place in a zip file.

In [ ]:
if not os.path.exists('my_custom_model_folder'):
    os.makedirs('my_custom_model_folder', exist_ok=True)

Change the labels (after `unlabeled`) as needed, adding all classes of objects here, one per line.  `unlabeled` just represents the background class.

In [ ]:
%%writefile my_custom_model_folder/labels.txt
unlabeled
mouse
keyboard
headphones

The following `config.json` is so that the Percept DK may find the labels and model (`.blob`) and need not be modified.

In [ ]:
%%writefile my_custom_model_folder/config.json
{
    "DomainType": "ssd100",
    "LabelFileName": "labels.txt",
    "ModelFileName": "ssdlite_mobilenet_v2.blob"
}

Copy the OpenVINO `.blob` model to a new folder.

In [ ]:
if os.path.exists(os.path.join('experiment_outputs','ssdlite_mobilenet_v2.blob')):
    shutil.move(os.path.join('experiment_outputs','ssdlite_mobilenet_v2.blob'), os.path.join('my_custom_model_folder','ssdlite_mobilenet_v2.blob'))

Create a zip file for the Percept DK Vision module to use with the `labels.txt`, `config.json` and the model `.blob` file inside of it.

In [ ]:
shutil.make_archive('my_custom_model', 'zip', 'my_custom_model_folder')

## Deploy the model to the Percept DK

1. In the [Azure Portal](https://portal.azure.com) navigate to your Storage Account.
2. Go to Containers under Data storage. 

![go to containers](article_images/azurestorage_containers.png)

3. Create a new container (this can be private with the use of SAS URLs for access or anonymous access/public).
4. Go to the container and upload the `my_custom_model.zip` file.
5. Click on the uploaded filename listed under the container and a new pane should open up with more properties.
6. If public/anonymous access, simply copy the URL under Properties and if private, generate a SAS URL and copy the Blob SAS URL.
7. In the Azure Portal navigate to your IoT Hub wherein the Percept DK edge device is specified.
8. Go to IoT Edge under Automatic Device Management.
9. Select your Percept DK device ID.
10. Select the `azureeyemodule` as follows.

![azureeyemodule](article_images/azureiothub_select_eye.png)

11. In the IoT Edge Module Details, select Module Identity Twin as follows.

![module identity twin](article_images/azureiothub_twin.png)

12. Update the `ModelZipUrl` value to hold the Blob URL created above as follows and press Save.

![update modelzipurl](article_images/azureiothub_modelzipurl.png)

This can be done programmatically as well.  See [this notebook](https://github.com/microsoft/azure-percept-advanced-development/blob/main/machine-learning-notebooks/transfer-learning/transfer-learning-using-ssd.ipynb) for an example of how to do this in Python (scroll to the OpenVINO section at the end).

## View the results

You will be able to view the results of deploying your new model at `https://<ip-address-of-your-device>:3000` as a live webstream.  You should see your object of interest in the web stream as follows as long as you have a good model (usually from carefully training on 100s of images per class).  The following model was created, for reference, with about 50 images per class (ideally, we would have much more data in the future).

<img src="article_images/percept_webstream.png" width="50%">

You may use a network stream player like [VLC](https://www.videolan.org/vlc/).  The RTSP to use in this case would be: `rtsp://ip-address-of-your-device:8554/result`.